In [1]:
from neurosynth import meta, decode, network, Dataset
import pandas
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from nilearn import plotting
import nilearn
import nibabel as nib
import pickle
%matplotlib inline

/Users/scotti.5/Documents/GitHub/pycortex/env/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [79]:
dataset = pickle.load(open("dataset.p","rb"))

In [33]:
dataset = Dataset('neurosynth-data/database.txt')

In [19]:
dataset.add_features('neurosynth-data/features.txt')

/Users/scotti.5/Documents/GitHub/pycortex/env/lib/python3.6/site-packages/neurosynth/base/dataset.py:624: FutureWarning: DataFrame/Series.to_dense is deprecated and will be removed in a future version
  old_data = self.data.to_dense()
/Users/scotti.5/Documents/GitHub/pycortex/env/lib/python3.6/site-packages/neurosynth/base/dataset.py:634: FutureWarning: DataFrame.to_sparse is deprecated and will be removed in a future version
  self.data = data.fillna(0.0).to_sparse()


In [49]:
terms = pandas.read_csv('neurosynth-data/analysis_filter_list.tsv',delimiter='\t')

In [94]:
kept_terms = terms['term'][terms['keep']==1]
np.savez("neurosynth_terms",'kept_terms')

In [13]:
for term in kept_terms:
    ids = dataset.get_studies(term)
    ma = meta.MetaAnalysis(dataset, ids)
    maps[term] = ma.images['association-test_z']
    ma.save_results(os.path.join('neurosynth-data','maps',term))

/Users/scotti.5/Documents/GitHub/pycortex/env/lib/python3.6/site-packages/neurosynth/base/dataset.py:716: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  feature_weights = self.data.ix[:, features]
/Users/scotti.5/Documents/GitHub/pycortex/env/lib/python3.6/site-packages/pandas/core/indexing.py:808: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
/Users/scotti.5/Documents/GitHub/pycortex/env/lib/python3.6/site-packages/pandas/core/frame.py:3451: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values 

## pickle concat_maps

In [67]:
def try_to_load_as_pickled_object_or_None(filepath):
    """
    This is a defensive way to write pickle.load, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    try:
        input_size = os.path.getsize(filepath)
        bytes_in = bytearray(0)
        with open(filepath, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        obj = pickle.loads(bytes_in)
    except:
        return None
    return obj

In [68]:
concat_maps = try_to_load_as_pickled_object_or_None("concat_maps.p")

In [166]:
for i in range(concat_maps.shape[0]):
    term = kept_terms.iloc[i];
    img = nilearn.image.new_img_like(dataset.masker.volume, dataset.masker.unmask(concat_maps[i],'array'))
    nib.save(img, os.path.join('term_maps',term))

# visualization

In [2]:
import cortex
subject = 'fsaverage'
anat = cortex.db.get_anat(subject, 'raw')
anat_data = anat.get_data()
anat_vol = cortex.Volume.empty(subject, 'atlas_2mm')

# cortex.webgl.show(data=anat_vol, template='my_template.html', recache=True)

In [27]:
from cortex.webgl.data import Package

In [170]:
for i in range(concat_maps.shape[0]):
    
    term = kept_terms.iloc[i];

    p = Package(cortex.Volume(("term_maps/"+term+".nii"), subject, 'atlas_2mm'))

    png = list(p.images.values())[0][0]

    with open(("neurosynth-maps/"+term+".png"),"wb") as f:
        f.write(png)

In [128]:
for i in range(10):
    p = Package(cortex.Volume((components[i,:,:,:].T/np.std(components[i,:,:,:].T)), subject, 'atlas_2mm'))

    png = list(p.images.values())[0][0]

    with open(("neurosynth-maps/pca"+str(i+1)+".png"),"wb") as f:
        f.write(png)

In [126]:
p = Package(rgbvol)

png = list(p.images.values())[0][0]

with open(("rgbvol3.png"),"wb") as f:
    f.write(png)

# PCA

In [8]:
comps = np.load('data/components.npz')['arr_0']
comps = comps.transpose(0,3,2,1)
comps.reshape(10,-1).shape

(10, 902629)

In [9]:
def make_colorvol(proj, meanstds, rgbpcs, mask, flips=tuple(), clip_lim=3):
    rgbdata = proj[rgbpcs,:]
    for f in flips:
        rgbdata[f] *= -1
    zrgb = (rgbdata.T / meanstds).T
    #zrgb = npp.rs(rgbdata.T).T
    crgb = np.clip(zrgb, -clip_lim, clip_lim) ## clip to 3 standard deviations
    srgb = crgb/clip_lim/2.0 + 0.5 ## scale to 0..1
    
    rgbvol = np.zeros([3]+list(mask.shape))
    rgbvol[0] = vox_to_mask(srgb[0], mask)
    rgbvol[1] = vox_to_mask(srgb[1], mask)
    rgbvol[2] = vox_to_mask(srgb[2], mask)

    colorvol = (rgbvol*255).astype(np.uint8)
    return colorvol.transpose(1,2,3,0)

def vox_to_mask(data, mask):
    dvol = mask.copy().astype(np.float)
    dvol[mask>0] = data
    return dvol

In [10]:
rescale = lambda x: x/x.std()

In [11]:
flat_comps = comps.reshape(10,-1)

colorvol = make_colorvol(flat_comps, flat_comps.std(1)[:3], [0,1,2], np.ones(comps[0].shape), clip_lim=4)

In [16]:
rgbvol = cortex.dataset.normalize((colorvol, 'fsaverage', 'atlas_2mm'))
rgbvol.alpha.vmin = 0

In [20]:
# cortex.webgl.show(data=rgbvol, template='my_template.html', recache=True)
cortex.webgl.make_static("static",data=rgbvol, template='my_template.html', with_labels=False, recache=True)

Generating new ctm file...
wm
wm
inflated
inflated


# individual PCAs

In [69]:
pca = PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
  svd_solver='randomized', tol=0.0, whiten=False).fit(concat_maps)
print(pca.explained_variance_ratio_)  
print(pca.singular_values_)

[0.10724451 0.07186315 0.04695774 0.03718591 0.03011859 0.02566107
 0.01888798 0.01386358 0.01315753 0.01177422]
[7371.67572625 6034.36808339 4877.89306864 4340.78108748 3906.57327303
 3605.9198214  3093.65168397 2650.43018536 2582.05702134 2442.55708149]


In [80]:
components = np.array([dataset.masker.unmask(pca.components_[i],output='array') for i in range(10)])
np.savez('components',components)

In [71]:
components = np.load("components.npz")
components

In [72]:
term_weights = pca.transform(concat_maps)

In [160]:
for i in range(10):
    a = term_weights[:,i].argsort()[::-1];
    print(np.concatenate((kept_terms.iloc[a[:5]], kept_terms.iloc[a[-5:]]), axis=None));
    print(i);

['motor' 'premotor' 'movements' 'movement' 'hand' 'medial' 'hippocampus'
 'medial prefrontal' 'emotional' 'amygdala']
0
['temporal' 'semantic' 'language' 'words' 'visual' 'striatum' 'insula'
 'somatosensory' 'motor' 'pain']
1
['auditory' 'superior temporal' 'speech' 'auditory cortex' 'sounds'
 'retrieval' 'task' 'prefrontal' 'memory' 'parietal']
2
['occipital' 'lateral occipital' 'visual' 'motion' 'intraparietal'
 'autobiographical' 'semantic' 'insula' 'auditory' 'temporal']
3
['hippocampus' 'amygdala' 'fusiform' 'faces' 'medial temporal'
 'inferior frontal' 'frontal gyrus' 'working memory' 'dorsolateral'
 'frontal']
4
['default' 'default mode' 'resting' 'resting state' 'dmn' 'nucleus'
 'insula' 'amygdala' 'striatum' 'reward']
5
['cerebellar' 'vi' 'cerebellum' 'hippocampal' 'hippocampus' 'insula'
 'medial prefrontal' 'junction' 'theory mind' 'social']
6
['auditory' 'auditory cortex' 'sounds' 'planum' 'planum temporale'
 'inferior frontal' 'words' 'semantic' 'reading' 'word']
7
['striat

## ATLAS

In [8]:
# load glasser atlas
lh_aparc_file = os.path.join('atlases','lh.HCP-MMP1.annot')
rh_aparc_file = os.path.join('atlases','rh.HCP-MMP1.annot')
lpinds, lpstats, lpnames = nib.freesurfer.read_annot(lh_aparc_file)
lpinds_orig, lpstats, lpnames = nib.freesurfer.read_annot(lh_aparc_file, True)
lpinds[lpinds_orig==0] = -1
rpinds, rpstats, rpnames = nib.freesurfer.read_annot(rh_aparc_file)
rpinds_orig, rpstats, rpnames = nib.freesurfer.read_annot(rh_aparc_file, True)
rpinds[rpinds_orig==0] = -1

aparc = cortex.Vertex(np.hstack([lpinds, rpinds]),'fsaverage')

Generating new ctm file...
wm
wm
inflated
inflated
Started server on port 9820


<JS: window.viewer>

In [355]:
np.hstack([lpinds, rpinds])

array([ 54,  49,  41, ..., 135, 135, 135], dtype=int32)